# Cleaning the Dataset

In [1]:
import json
with open('documents.json', 'r') as f:
    documents = json.load(f)


In [2]:
import os

if not os.path.exists('persistDirectory'):
    os.makedirs('persistDirectory')

In [13]:
documents[0]['directions']

['"preheat oven to 350 degree."',
 '"grease a 9-inch pie pan. saute onion in 1 tablespoon butter."',
 '"mix all ingredients together and pour into pie pan."',
 '"bake for about 1/2 hour."']

In [3]:
### preprocessing the documents 
def clean_shape_documents(documents):           
    cleanedDcos = list()
    for doc in documents:       #a doc is a dictionary {title:'',ingredients:'['','','']',directions:''}
        cleanedTitle = doc['title'].lower().strip().replace('-', ' ')
        cleanedIngredients = [ing.strip().replace('"','').replace('c.','cups') for ing in (ings for ings in doc['ingredients'])]
        cleanedDirections = [dir.strip().replace('"','') for dir in (dirs for dirs in doc['directions'])]

        cleanedDcos.append({'title':cleanedTitle, 'ingredients':cleanedIngredients, 'directions':cleanedDirections})
    return cleanedDcos

cleanedDocuments = clean_shape_documents(documents)

In [15]:
cleanedDocuments[0] 

{'title': 'zucchini pie',
 'ingredients': ['2 small zucchini',
  'diced (about 3 to 4 cups)',
  '1 large onion',
  'diced',
  '1 cups bisquick',
  '3 eggs',
  'beaten',
  '1/2 cups vegetable oil',
  '1 cups parmesan cheese',
  'pepper to taste',
  'parsley to taste',
  '1 tbsp. butter'],
 'directions': ['preheat oven to 350 degree.',
  'grease a 9-inch pie pan. saute onion in 1 tablespoon butter.',
  'mix all ingredients together and pour into pie pan.',
  'bake for about 1/2 hour.']}

In [41]:
cleanedDocuments[2]

{'title': 'succotash salad',
 'ingredients': ['salad:',
  '1 (16-ounce) bag frozen baby lima beans',
  '3 cups fresh corn kernels',
  '1 cup chopped red bell pepper',
  '3/4 cup chopped green onion',
  '1/2 cup finely chopped red onion',
  '1/4 cup chopped fresh flat-leaf parsley',
  '2 tablespoons chopped fresh oregano',
  'dressing:',
  '1/3 cup fresh lemon juice',
  '2 tablespoons dijon mustard',
  '2 tablespoons olive oil',
  '3/4 teaspoon salt',
  '3/4 teaspoon freshly ground black pepper'],
 'directions': ['to prepare salad',
  'cook beans in boiling water for 12 minutes. drain; rinse with cold water. drain. combine beans',
  'corn',
  'and next 5 ingredients (corn through oregano).',
  'to prepare dressing',
  'combine lemon juice and remaining ingredients',
  'stirring with a whisk. drizzle over salad',
  'and toss to coat.']}

In [10]:
#creating document using the langchain documnet class
from langchain_core.documents import Document
dataset1 = [Document(page_content='title: '+title+'\ningredients: '+str(ingredients).replace('[','').replace(']','')+'\ndirections: '+str(directions).replace('[','').replace(']','')) for title,ingredients,directions in (doc.values() for doc in cleanedDocuments)]

In [11]:
#creating document using the langchain documnet class
from langchain_core.documents import Document
dataset2 = [Document(page_content='title: '+title+' ingredients: '+str(ingredients).replace('[','').replace(']','')+' directions: '+str(directions).replace('[','').replace(']','')) for title,ingredients,directions in (doc.values() for doc in cleanedDocuments)]

In [4]:
#creating document using the langchain documnet class
from langchain_core.documents import Document
dataset3 = [Document(page_content=title+str(ingredients).replace('[','').replace(']','')+str(directions).replace('[','').replace(']','')) for title,ingredients,directions in (doc.values() for doc in cleanedDocuments)]

# Storing The Data (NEW)

In [27]:
!mkdir phidbb1

In [28]:
!mkdir phidbb2

In [29]:
!mkdir phidbb3

In [30]:
# Import required modules
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import streamlit as st
import os
import time
from tqdm import tqdm
datasets = [dataset1, dataset2, dataset3]
phiEmbedder = OllamaEmbeddings(model="phi3",show_progress=True)
for i,dataset in tqdm(enumerate(datasets)):
    vector_db = Chroma.from_documents(
        documents=dataset[0:300], 
        embedding=phiEmbedder,
        collection_name="recipe",
        persist_directory=f"phidbb{i+1}"
    )


OllamaEmbeddings: 100%|██████████| 300/300 [49:16<00:00,  9.85s/it]
3it [2:30:24, 3008.13s/it]


In [5]:
len(dataset3)

55778

In [7]:
# Import required modules
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma

from tqdm import tqdm

phiEmbedder = OllamaEmbeddings(model="phi3",show_progress=True)

vector_db = Chroma.from_documents(
    documents=dataset3[0:3000], 
    embedding=phiEmbedder,
    collection_name="recipe",
    persist_directory=f"Mega"
)


OllamaEmbeddings: 100%|██████████| 3000/3000 [7:38:04<00:00,  9.16s/it]   


# Reading from Database


In [1]:
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.embeddings.ollama import OllamaEmbeddings

def readChromadb(folderPath:str , model:str , collection_name:str):
    embedder = OllamaEmbeddings(model=model,show_progress=True)
    vectorDB = Chroma(persist_directory=folderPath,collection_name=collection_name,embedding_function=embedder)
    return vectorDB

#reading all the 3 vector databases
phidbb1 = readChromadb(folderPath='phidbb1',model='phi3',collection_name='recipe')
phidbb2 = readChromadb(folderPath='phidbb2',model='phi3',collection_name='recipe')
phidbb3 = readChromadb(folderPath='phidbb3',model='phi3',collection_name='recipe')

In [8]:
type(phidbb1)

langchain_community.vectorstores.chroma.Chroma

# Querying (testing and experimenting)

In [38]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

def makeOllamaQuery(vectorDB , model:str , numResults:int, promptQuery , ragPromptTemplate):
    llm = ChatOllama(model = model)
    retriever = MultiQueryRetriever.from_llm(vectorDB.as_retriever(num_results=numResults), llm=llm,prompt=promptQuery)
    prompt = ChatPromptTemplate.from_template(ragPromptTemplate)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    userInp = input("")
    if userInp == '':
        return None
    results = chain.invoke(userInp)
    return results

In [3]:
promptQuery = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI chef assistant trained on a RAG process. Your task is to take in the users recipe name and then generate
    the ingredients need and directions to cook that recipe. Only include the title,ingredients and directions you find in the vector database. 
    Use the Follow Format to print the results
    Recipe Name: , Ingredients: , Directions:
    User question: {question}""",
)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

numResults = 5      #this will allow me to check and compare top 5 most similar results
model='phi3'



In [31]:
lol = makeOllamaQuery(vectorDB=phidbb1,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)


In [36]:
lDocs=lol.invoke('zucchini pie')

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.58s/it]


In [12]:
#testing the 'zucchini pie' recpie, the original recipe is as follows:
cleanedDocuments[0]

{'title': 'zucchini pie',
 'ingredients': ['2 small zucchini',
  'diced (about 3 to 4 cups)',
  '1 large onion',
  'diced',
  '1 cups bisquick',
  '3 eggs',
  'beaten',
  '1/2 cups vegetable oil',
  '1 cups parmesan cheese',
  'pepper to taste',
  'parsley to taste',
  '1 tbsp. butter'],
 'directions': ['preheat oven to 350 degree.',
  'grease a 9-inch pie pan. saute onion in 1 tablespoon butter.',
  'mix all ingredients together and pour into pie pan.',
  'bake for about 1/2 hour.']}

## phidbb1
### sending input 'zucchini pie'

In [4]:
#making query on the embedding which have title, ingredients, directions keywords and \n seperating eveything

phidbb1Result1 = makeOllamaQuery(vectorDB=phidbb1,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb1Result1)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


To create a Zucchini Pie, you will need the following ingredients and follow these steps:

Ingredients:
- 4 cups grated zucchini (about 3 medium)
- 1/2 cup granulated sugar
- 1/2 teaspoon salt
- 1 large egg
- 1/2 cup all-purpose flour
- 1/4 cup milk or buttermilk
- 1 tablespoon vegetable oil
- 1 teaspoon vanilla extract
- Pie crust (store-bought or homemade) for the bottom and top of the pie

Directions:

1. Preheat your oven to 375°F (190°C).
2. Grate zucchini, then line a colander with a clean kitchen towel or paper towels. Press out excess moisture from the grated zucchini by layering it in the cloth and squeezing gently. This helps prevent the pie from becoming too soggy.
3. In a bowl, combine 1/2 cup sugar and salt with the drained zucchini. Mix well to coat the zucchini evenly. Set aside for about 5-10 minutes until it starts releasing its liquid. After that time, strain the excess moisture using a colander or clean kitchen cloth again if necessary.
4. In another bowl, combine th

### sending input 'title: zucchini pie'

In [5]:
phidbb1Result2 = makeOllamaQuery(vectorDB=phidbb1,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb1Result2)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


To create Zucchini Pie, you will need to start by preparing the following ingredients:

Ingredients:
- 4 cups grated zucchini (fresh or from frozen)
- 1 cup granulated sugar
- 2 large eggs
- 1/2 teaspoon salt
- 1 tablespoon lemon juice
- 1 tablespoon melted butter or margarine
- For the crust:
    - 2 and 3/4 cups all-purpose flour
    - 1 cup (2 sticks) unsalted butter, chilled and diced into small cubes
    - 1 teaspoon salt
    - 1 tablespoon sugar
    - 6 to 8 tablespoons ice cold water
  
Directions:

1. Preheat your oven to 375°F (190°C).
2. Start by making the crust: In a large mixing bowl, combine flour, salt, and sugar. Add in chilled butter cubes and mix until the mixture resembles coarse crumbs using your fingers or pastry cutter. Gradually add ice cold water while gently combining to form a dough. Divide the dough into two equal portions - one for bottom crust, and another smaller portion for top crust (if desired).
3. On a lightly floured surface, roll out the larger dough

## phidbb2
### same tests

In [6]:
#making query on the embedding which have title, ingredients, directions keywords and \n seperating eveything

phidbb2Result1 = makeOllamaQuery(vectorDB=phidbb2,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb2Result1)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


To create a zucchini pie, you would need to adjust the ingredients list provided for any of the existing recipes or combine elements from different ones. Since there's no direct zucchini pie recipe in your examples, here is an estimated recipe using similar techniques and measurements:

Ingredients:
- 2 cups grated zucchini
- 1 cup sugar (adjust to taste)
- 3 eggs
- 1/4 cup flour
- 1 tsp. cinnamon
- 1/4 tsp. nutmeg
- 1/2 tsp. salt
- 1/4 cup milk or buttermilk (adjust as needed)
- 1 pie crust, pre-baked and cooled
- Optional: 1/2 cup chopped nuts (e.g., walnuts), for topping

Directions:
1. Preheat your oven to 350 degrees Fahrenheit (175 degrees Celsius).
2. Grate the zucchini and press out excess moisture using a clean kitchen towel or cheesecloth. Set aside.
3. In a mixing bowl, combine sugar, eggs, flour, cinnamon, nutmeg, salt, and milk until well mixed.
4. Fold in the grated zucchini into your mixture until evenly distributed.
5. Pour the filling into the pre-baked pie crust, smoo

In [7]:
# title: zucchini pie
phidbb2Result2 = makeOllamaQuery(vectorDB=phidbb2,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb2Result2)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.53s/it]


Title: Zesty Vegetarian Zucchini Pie Ingredients and Cooking Instructions

Ingredients:

- 3 medium zucchinis, grated (about 4 cups)

- 1/2 cup all-purpose flour

- 1 tsp. baking powder

- A pinch of salt and pepper

- 1 large egg

- 1/3 cup brown sugar

- 1/4 cup vegetable oil

- 1/2 tsp. vanilla extract

- 1 cup shredded mozzarella cheese (optional)

- 1/4 cup finely chopped fresh parsley for garnish


Directions:

1. Preheat your oven to 375°F (190°C). Lightly grease a pie dish with nonstick cooking spray or butter.

2. In a large bowl, combine the grated zucchinis and flour, baking powder, salt, and pepper. Mix until well incorporated.

3. In another bowl, beat the egg, brown sugar, vegetable oil, and vanilla extract together until smoothly blended.

4. Fold the dry ingredients into the wet mixture until a consistent batter forms. Gently fold in the shredded mozzarella cheese if using.

5. Transfer the zucchini pie filling to the prepared dish, spreading it evenly.

6. Bake for 40-

## phidbb3

In [8]:
phidbb3Result1 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb3Result1)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


To make zucchini pie, you will need to start by preheating your oven. Then follow this recipe:

Ingredients:
- 1 large zucchini (about 2 cups grated)
- 3 eggs
- 3/4 cup granulated sugar
- 1/4 cup all-purpose flour
- 1 tsp vanilla extract
- 1/2 tsp ground cinnamon
- 1/4 tsp salt
- 1 (9 inch) unbaked pie crust

Instructions:

1. Preheat your oven to 375°F (190°C).
2. Grate the zucchini and set aside, allowing any excess moisture to drain out by wrapping it in a clean kitchen towel or paper towels for about 10 minutes before moving on. This helps avoid a soggy pie crust.
3. In a large mixing bowl, combine eggs, sugar, flour, vanilla extract, cinnamon, and salt. Mix well until smooth.
4. Add the grated zucchini to your wet ingredients and stir gently to incorporate all of it evenly.
5. Pour your filling into the unbaked pie crust and distribute it evenly.
6. Bake in preheated oven for about 25-30 minutes, or until a knife inserted into the center comes out clean. The edges may turn golden 

In [9]:
phidbb3Result2 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb3Result2)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.64s/it]


**Zesty Zucchini Pie Recipe**

*Ingredients:*

- 2 cups finely grated zucchini

- 1/2 cup all-purpose flour

- 1 tablespoon sugar

- 1/4 teaspoon salt

- 1/8 teasbook_tsp nutmeg (freshly grated)

- 2 large eggs

- 1/3 cup whole milk

- 2 tablespoons unsalted butter, melted

- 1 pre-made pie crust or homemade pie dough

- Egg wash (1 egg beaten with 1 tablespoon water) for brushing the crust


*Directions:*

1. Preheat your oven to 375°F (190°C).

2. If using homemade pie dough, roll out and fit into a 9-inch pie pan or line with pre-made crust. Trim any excess hanging over the edges. Refrinate


3. In a large bowl, mix together grated zucchini, flour, sugar, salt, nutmeg, eggs, and melted butter until just combined.

4. Pour the zucchini mixture into the prepared pie crust, smoothing out the top with a spatula.

5. Cover the edges of the pie with foil to prevent over-browning if needed.

6. Brush the top and sides of the pie with egg wash for added color.

7. Bake in the preheated oven

## Applying Cosine Similarity to see how much similar they are to each other and with the original results

In [17]:
cleanedDocuments[0]

{'title': 'zucchini pie',
 'ingredients': ['2 small zucchini',
  'diced (about 3 to 4 cups)',
  '1 large onion',
  'diced',
  '1 cups bisquick',
  '3 eggs',
  'beaten',
  '1/2 cups vegetable oil',
  '1 cups parmesan cheese',
  'pepper to taste',
  'parsley to taste',
  '1 tbsp. butter'],
 'directions': ['preheat oven to 350 degree.',
  'grease a 9-inch pie pan. saute onion in 1 tablespoon butter.',
  'mix all ingredients together and pour into pie pan.',
  'bake for about 1/2 hour.']}

In [18]:
originalResult = ""
for key, value in cleanedDocuments[0].items():
    originalResult += f"{key}: {value}\n"


In [19]:
originalResult

"title: zucchini pie\ningredients: ['2 small zucchini', 'diced (about 3 to 4 cups)', '1 large onion', 'diced', '1 cups bisquick', '3 eggs', 'beaten', '1/2 cups vegetable oil', '1 cups parmesan cheese', 'pepper to taste', 'parsley to taste', '1 tbsp. butter']\ndirections: ['preheat oven to 350 degree.', 'grease a 9-inch pie pan. saute onion in 1 tablespoon butter.', 'mix all ingredients together and pour into pie pan.', 'bake for about 1/2 hour.']\n"

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

variables = [originalResult, phidbb1Result1, phidbb1Result2, phidbb2Result1, phidbb2Result2, phidbb3Result1, phidbb3Result2]
variable_names = ['originalResult', 'phidbb1Result1', 'phidbb1Result2', 'phidbb2Result1', 'phidbb2Result2', 'phidbb3Result1', 'phidbb3Result2']

vectorizer = CountVectorizer().fit_transform(variables)

#calculate the cosine similarity between each pair of variables
cosine_similarities = cosine_similarity(vectorizer)

#results
for i in range(len(variables)):
    for j in range(i + 1, len(variables)):
        print(f"{variable_names[i]} vs {variable_names[j]}: {cosine_similarities[i][j]}")
    print('='*100)



originalResult vs phidbb1Result1: 0.32201349793031875
originalResult vs phidbb1Result2: 0.40833713578635994
originalResult vs phidbb2Result1: 0.3581934413376319
originalResult vs phidbb2Result2: 0.3679498519182043
originalResult vs phidbb3Result1: 0.4633686276961576
originalResult vs phidbb3Result2: 0.38801159182646217
phidbb1Result1 vs phidbb1Result2: 0.8252220334858131
phidbb1Result1 vs phidbb2Result1: 0.7900233448105534
phidbb1Result1 vs phidbb2Result2: 0.7532117443016427
phidbb1Result1 vs phidbb3Result1: 0.6995680682079951
phidbb1Result1 vs phidbb3Result2: 0.8289385818779827
phidbb1Result2 vs phidbb2Result1: 0.7918070206376462
phidbb1Result2 vs phidbb2Result2: 0.7090607554337315
phidbb1Result2 vs phidbb3Result1: 0.7305773871655125
phidbb1Result2 vs phidbb3Result2: 0.7693995676855514
phidbb2Result1 vs phidbb2Result2: 0.7241184674119164
phidbb2Result1 vs phidbb3Result1: 0.7299342357174551
phidbb2Result1 vs phidbb3Result2: 0.750744386453635
phidbb2Result2 vs phidbb3Result1: 0.64604478

since the 3rd database with the input 'zucchini pie' worked so well, lets take 3 more trials from it and average its results

In [27]:
r1 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(r1)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]


To make Zucchini Pie, you'll need to gather ingredients such as grated zucchini, flour, sugar, eggs, milk or buttermilk, and spices like cinnamon. Here is a basic recipe:

1. Preheat the oven to 375°F (190°C).
2. Grate about 4 cups of zucchini and squeeze out excess moisture using paper towels or cheesecloth.
3. In a large mixing bowl, combine the grated zucchini with 1 cup sugar, 1 teaspoon cinnamon, and a pinch of nutmeg (optional). Set aside for about 5-10 minutes to let flavors meld together.
4. Prepare your crust by combining 2 cups all-purpose flour with 1/2 cup sugar, and a teaspoon of salt in a large bowl. Cut in 1 cup cold unsalted butter using a pastry cutter or fork until the mixture resembles coarse meal. Slowly add ice water (about 4-6 tablespoons) to form a dough, and then shape it into a disc.
5. Roll out your pie crust on parchment paper between two sheets of plastic wrap until about 12 inches in diameter. Transfer the rolled dough into a 9-inch pie plate. Press down ge

In [26]:
r2 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(r2)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


To make a zucchini pie, you will need to start by preparing the zucchini filling. Here's how you can do it:

Ingredients:
- 2 medium zucchinis (about 1 lb.), grated and drained of excess moisture
- 1/4 cup white sugar
- 1 tsp vanilla extract
- 2 large eggs, beaten
- 1 tbsp all-purpose flour
- Salt and pepper to taste
- For the crust:
- 1 package (9 inches) refrigerated pie dough
- Additional sugar for sprinkling on top

Directions:

1. Preheat your oven to 375°F (190°C).
2. In a large bowl, combine the grated zucchinis, white sugar, vanilla extract, beaten eggs, flour, salt, and pepper until well mixed. Stir in thoroughly with a wooden spoon after each addition to ensure even distribution of ingredients.
3. Roll out one sheet of pie dough on a lightly-floured surface into a 12-inch circle. Transfer the crust to your pie pan, gently pressing it against the bottom and sides. Trim any excess overhang.
4. Pour the zucchini filling into the prepared pie crust, distributing it evenly with a 

In [28]:
r3 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(r3)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.64s/it]


To make a delicious Zucchini Pie, you'll want to follow this recipe which combines classic ingredients with the freshness of grated zucchini. Here's how to do it:

Ingredients:
- 1 lb. zucchini (grated)
- 2 eggs
- 1/2 cup sugar
- 1 tsp. vanilla extract
- 3 cups all-purpose flour
- 1 tsp. salt
- 1 tbsp. baking powder
- 4 oz. (8 tsp.) unsalted butter, chilled and diced
- 1/2 cup milk
- Pie crust (homemade or store-bought)

Instructions:
1. Preheat your oven to 375°F (190°C).
2. In a large bowl, combine the grated zucchini with eggs and sugar until well mixed. Add vanilla extract and set aside.
3. For the pie crust: In another bowl, mix flour, salt, and baking powder. Cut in cold butter using your fingers or a pastry cutter until the mixture resembles coarse meal. Gradually add milk to form a dough. Roll out the dough between two sheets of parchment paper into a circle that's approximately 12-inches wide, and fit it into your pie plate.
4. Pour the zucchini filling evenly over the rolled-

In [29]:
variables = [originalResult, phidbb3Result1, r1, r2, r3]
variable_names = ['originalResult', 'phidbb3Result1', 'r1', 'r2', 'r3']

vectorizer = CountVectorizer().fit_transform(variables)

#calculate the cosine similarity between each pair of variables
cosine_similarities = cosine_similarity(vectorizer)
# Find the index of phidbb3Result1
phidbb3_index = variable_names.index('phidbb3Result1')
# Find the indices of r1, r2, r3
r_indices = [variable_names.index('r1'), variable_names.index('r2'), variable_names.index('r3')]
# Calculate the average similarity between phidbb3Result1 and its repetitions with originalResult
avg_similarity = 0
for r_idx in r_indices:
    avg_similarity += cosine_similarities[phidbb3_index][r_idx]
avg_similarity /= len(r_indices)

print(f"The average cosine similarity between phidbb3Result1 and its repetitions with originalResult is: {avg_similarity}")


The average cosine similarity between phidbb3Result1 and its repetitions with originalResult is: 0.7424861712485405


In [6]:
# from langchain.prompts import ChatPromptTemplate, PromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_community.chat_models import ChatOllama
# from langchain_core.runnables import RunnablePassthrough
# from langchain.retrievers.multi_query import MultiQueryRetriever

# # LLM from Ollama
# local_model = "phi3"
# llm = ChatOllama(model=local_model)

# QUERY_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""You are an AI chef assistant. Your task is to take in the users recipe name and then generate
#     the ingredients need and directions to cook that recipe. Use the Follow Format to print the results
#     Recipe Name: , Ingredients: , Directions:
#     User question: {question}""",
# )

# retriever = MultiQueryRetriever.from_llm(
#     vector_db.as_retriever(num_results=5), 
#     llm,
#     prompt=QUERY_PROMPT
# )

# # RAG prompt
# template = """Answer the question based ONLY on the following context:
# {context}
# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# results = chain.invoke(input(""))

NameError: name 'vector_db' is not defined

In [39]:
phidbb3Result2 = makeOllamaQuery(vectorDB=phidbb3,model=model,numResults=numResults,promptQuery=promptQuery,ragPromptTemplate=template)
print(phidbb3Result2)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.43s/it]


With eggs, oil, water, onions, sugar, and salt, you can make a classic dish like scrambled eggs. Here is a simple recipe for "Scrambled Eggs" that incorporates all your ingredients:

Ingredients:
- 4 large eggs
- 2 tablespoons of oil (olive oil or butter)
- Salt, to taste
- Pepper, to taste
- 1/4 cup chopped onions
- 1 teaspoon sugar
- Water (if needed for consistency)

Instructions:
1. Heat the oil in a non-stick skillet over medium heat. Add the chopped onion and sauté until translucent, about 2 to 3 minutes.
2. Sprinkle sugar into the skillet and continue cooking for an additional minute or so. This will help caramelize the onions slightly while adding a hint of sweetness.
3. In a mixing bowl, whisk together the eggs with salt and pepper to taste until well combined. If needed, add small amounts of water (about 1-2 tablespoons) for desired consistency.
4. Pour the beaten egg mixture into the skillet over the onions. Let it cook without stirring for a few seconds until the edges star

# Text to Speech testing

In [42]:
from transformers import pipeline

pipe = pipeline("text-to-speech", model="suno/bark-small")
text = "[clears throat] This is a test ... and I just took a long pause."
output = pipe(text)

C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abdul\.cache\huggingface\hub\models--suno--bark-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

c:\Users\abdul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
c:\Users\abdul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\transformers\models\encodec\modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

...

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [43]:
from IPython.display import Audio
Audio(output["audio"], rate=output["sampling_rate"])

In [44]:
from transformers import pipeline

pipe = pipeline("text-to-speech", model="suno/bark-small")
text="some text"
output = pipe(text)

C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\abdul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
C:\Users\abdul\AppData\Roaming\Python\Python311\site-packages\transformers\models\encodec\modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.te

In [45]:
from IPython.display import Audio
Audio(output["audio"], rate=output["sampling_rate"])